# 1. Import các thư viện cần thiết

In [1]:
import os
import playsound as ps
import speech_recognition as sr
import time
import sys
import ctypes
import wikipedia
import datetime
import json
import re
import webbrowser
import smtplib
import requests
import urllib
import urllib.request as urllib2
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from time import strftime
from gtts import gTTS
from youtube_search import YoutubeSearch

# 2. Khai báo biến mặc định

In [2]:
wikipedia.set_lang('vi')
language = 'vi'
path = ChromeDriverManager().install()



====== WebDriver manager ======
Current google-chrome version is 90.0.4430
Get LATEST driver version for 90.0.4430
Get LATEST driver version for 90.0.4430
Trying to download new driver from https://chromedriver.storage.googleapis.com/90.0.4430.24/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\phamd\.wdm\drivers\chromedriver\win32\90.0.4430.24]


# 3. Chức năng chuyển văn bản thành âm thanh

  -  Dùng `gTSS (google Text To Speech)` chuyển văn bản thành âm thanh theo tiếng Việt và lưu lại dưới dạng file **sound.mp3**
  -  Sau đó dùng hàm `playsound.playsound()` để đọc file **sound.mp3** trên máy tính
  -  Xóa file **sound.mp3** sau khi đọc xong để tránh lỗi trùng lắp file khi đọc nhiều văn bản

In [25]:
def speak(text):
    print("Vietvias: {}".format(text))
    tts = gTTS(text=text, lang='vi', slow=False)
    tts.save("sound.mp3")
    ps.playsound("sound.mp3")
    os.remove("sound.mp3")

# 4. Chức năng chuyển âm thanh thành văn bản
   Trong chức năng này, dùng 2 hàm là `get_audio()`, `stop()` và `get_text()`.
   
   
### Trong hàm `get_audio()`
 -  Dùng thư viện speech_recognition có chức năng nhân diện giọng nói để chuyển âm thanh thành văn bản.
 -  Âm thanh được đọc vào sau đó được xử lý qua hàm `listen()` và lưu vào biến **audio**.
 -  Âm thanh sẽ được nhận dạng ở tiếng Việt bằng hàm `recognize_google()` và lưu kết quả vào biến **text**
 - Nếu **audio** không lỗi tức `recognize_google()` có thể nhận dạng được thì hàm sẽ trả về **text**. Ngược lại, nếu lỗi thì sẽ trả về **0**.
 
  ### Trong hàm `stop()`
   Đọc đoạn text "Tạm biệt" bằng cách dùng hàm `speak()` ở trên.
   
  ### Trong hàm `get_text()`
 - Máy tính sẽ nhận dạng âm thanh của người đọc tối đa 3 lần (bằng cách dùng for 3 lần), nếu text khác **0** thì `get_text()` trả vè `text.lower()`. 
 - Ngược lại text = 0 thì trả về hàm `stop()` và `get_text()` trả về 0
 - `time.sleep(2)`: tạm dừng 2s tránh máy đọc các đoạn văn bản bị khớp nhau. 


In [21]:
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Tôi: ", end='')
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio, language="vi-VN")
            print(text)
            return text
        except:
            print("...")
            return 0
        
def stop():
    speak("Tạm biệt")
    
def get_text():
    for i in range(3):
        text = get_audio()
        if text:
            return text.lower()
        elif i < 2:
            speak("Vietvias không nghe rõ. Bạn nói lại được không!")
    time.sleep(2)
    stop()
    return 0

# 5. Chức năng giao tiếp, chào hỏi
- Chức năng giao tiếp thông thường giữa người và máy bằng hàm `hello(name)'.
- Dùng biến **day_time** lưu giờ hiện tại sau đó.

In [5]:
def hello(name):
    day_time = int(strftime('%H'))
    if day_time < 12:
        speak("Chào buổi sáng {}. Chúc bạn một ngày tốt lành.".format(name))
    elif 12 <= day_time < 18:
        speak("Chào buổi chiều {}. Bạn đã dự định gì cho chiều nay chưa?".format(name))
    else:
        speak("Chào buổi tối {}. Bạn đã ăn tối chưa nhỉ?".format(name))

# 6. Chức năng hiển thị thời gian
- Dùng thư viện **`datetime`**, lưu vào biến **now**
- Nếu trong biến **text** đầu vào có từ `"giờ"` thì sẽ đọc thời gian hiện tại trong ngày
- Nếu trong biến **text** đầu vào có từ `"ngày"` thì sẽ đọc thời điểm hiện tại trong năm

In [6]:
def get_time(text):
    now = datetime.datetime.now()
    if "giờ" in text:
        speak('Bây giờ là %d giờ %d phút' % (now.hour, now.minute))
    elif "ngày" in text:
        speak("Hôm nay là ngày %d tháng %d năm %d" %
              (now.day, now.month, now.year))
    else:
        speak("Vietvias chưa hiểu ý của bạn. Bạn nói lại được không?")

# 7. Chức năng mở ứng dụng hệ thống, website và chức năng tìm kiếm từ khóa trên Google
### Hàm `open_application()`
- Dùng hàm `os.startfile()` để mở các file ứng dụng từ hệ thống

### Hàm `open_website()`
- Dùng hàm `re.search()` để tách "domain" sau từ "mở" trong biến **text** và ghép nó với phần tiền tố "https://www." để tạo thành **url** của web.
- Tiếp theo, dùng `webbroser.open(url)` để mở trang web theo url
- Nếu domain được tìm thấy thì thực hiện mở website bằng `open_website()` và trả về **True**, ngược lại thì không thực hiện gì cả và trả về **False**.

### Hàm `open_google_and_search()`
- Tách từ khóa sau từ "kiếm" trong **text** bằng `split()` rồi lưu vào **search_for**
- Gọi hàm `webdriver.Chrome(path)` để mở Google Chrome 
- Dùng hàm `driver.find_element_by_path()` để lấy thẻ **query** (viết tắt là q) rồi lưu vào biến que
- Biến **que** thực hiện tiềm kiếm từ khóa **search_for** và trả trên Google Search. 

In [7]:
def open_application(text):
    if "google" in text:
        speak("Mở Google Chrome")
        os.startfile('C:\Program Files (x86)\Google\Chrome\Application\chrome.exe')
    elif "word" in text:
        speak("Mở Microsoft Word")
        os.startfile('C:\Program Files\Microsoft Office\\root\Office16\\WINWORD.EXE')
    elif "excel" in text:
        speak("Mở Microsoft Excel")
        os.startfile('C:\Program Files\Microsoft Office\\root\Office16\EXCEL.EXE')
    else:
        speak("Ứng dụng chưa được cài đặt. Bạn hãy thử lại!")
        
def open_website(text):
    reg_ex = re.search('mở (.+)', text)
    if reg_ex:
        domain = reg_ex.group(1)
        url = 'https://www.' + domain
        webbrowser.open(url)
        speak("Trang web bạn yêu cầu đã được mở.")
        return True
    else:
        return False
    
def open_google_and_search(text):
    search_for = text.split("kiếm", 1)[1]
    speak('Okay!')
    driver = webdriver.Chrome(path)
    driver.get("http://www.google.com")
    que = driver.find_element_by_xpath("//input[@name='q']")
    que.send_keys(str(search_for))
    que.send_keys(Keys.RETURN)

# 8. Chức năng gửi Email (chưa hoàn thiện)
- Sử dụng thư viện **`smtplib`** để thực hiện gửi mail bằng stmp.
- SMTP - Simple Mail Transfer Protocol: Giao thức truyền tải thư tín đơn giản
- Lấy tên người nhận bằng hàm `get_text()`, lưu vào biến **recipient**
    vd: tìm thấy tên "thục" trong tên người cần gửi thì thực hiện gửi Email.
- Gọi `get_text()` để lấy nội dung cần gửi rồi lưu vào biến **content**
- Tiếp tục mở đường truyền gửi Email băng `stmp` rồi đăng nhập tài khoản Gmail của mình
- Kết nối thành công, sẽ gửi email từ địa chỉ đã đăng nhập đến địa chỉ người nhận (dinhthucf12@gmail.com) với nội dung **content** rồi đóng đường truyền

In [8]:
def send_email(text):
    speak('Bạn gửi email cho ai nhỉ')
    recipient = get_text()
    if 'thục' in recipient:
        speak('Nội dung bạn muốn gửi là gì')
        content = get_text()
        mail = smtplib.SMTP('smtp.gmail.com', 587)
        mail.ehlo()
        mail.starttls()
        mail.login('phamdinhthuc020100@gmail.com', 'dinhthuc123')
        mail.sendmail('phamdinhthuc020100@gmail.com',
                      'dinhthucf12@gmail.com', content.encode('utf-8'))
        mail.close()
        speak('Email của bạn vùa được gửi. Bạn check lại email nhé.')
    else:
        speak('Vietvias không hiểu bạn muốn gửi email cho ai. Bạn nói lại được không?')

# 9. Chức năng xem dự báo thời tiết
- Dùng **Open WeatherMap** tại trang web `openweathermap.org`
- Dùng biến ow_url để lưu đường dẫn đến api của trang web
- Gọi get_text() để lấy thông tin thành phố cần xem thông tin thời tiết lưu vào biến **city**. Nếu không nghe được tên thì sẽ bỏ qua
- Kết nối với api của trang web (cần đăng ký tài khoản để lấy api_key)
- Biến **call_url** sẽ lưu đường dẫn đầy đủ để truy vấn bao gồm thông tin tên thành phố **city**
- Gọi `requests.get(call_url)` lấy thông tin truy vấn, lưu vào **response**
- `response.json()` sẽ chuyển dữ liệu về json rồi lưu vào **data**
- Nếu data["cod"] không trả về 404 tức requests không lỗi, thực hiện chức năng
- Từ **data** lấy các thông tin: **temperature**, **pressure**, **humidity**
- Lưu các kết quả vào biến **content** và đọc kết quả
- time.sleep(20): tạm dừng 20s để đọc **content**


In [9]:
def current_weather():
    speak("Bạn muốn xem thời tiết ở đâu?")
    ow_url = "http://api.openweathermap.org/data/2.5/weather?"
    city = get_text()
    if not city:
        pass
    api_key = "fe8d8c65cf345889139d8e545f57819a"
    call_url = ow_url + "appid=" + api_key + "&q=" + city + "&units=metric"
    response = requests.get(call_url)
    data = response.json()
    if data["cod"] != "404":
        city_res = data["main"]
        current_temperature = city_res["temp"]
        current_pressure = city_res["pressure"]
        current_humidity = city_res["humidity"]
        suntime = data["sys"]
        sunrise = datetime.datetime.fromtimestamp(suntime["sunrise"])
        sunset = datetime.datetime.fromtimestamp(suntime["sunset"])
        wthr = data["weather"]
        weather_description = wthr[0]["description"]
        now = datetime.datetime.now()
        content = """
        Hôm nay là ngày {day} tháng {month} năm {year}
        Mặt trời mọc vào {hourrise} giờ {minrise} phút
        Mặt trời lặn vào {hourset} giờ {minset} phút
        Nhiệt độ trung bình là {temp} độ C
        Áp suất không khí là {pressure} héc tơ Pascal
        Độ ẩm là {humidity}%
        Trời hôm nay quang mây. Dự báo mưa rải rác ở một số nơi.""".format(day = now.day,month = now.month, year= now.year, hourrise = sunrise.hour, minrise = sunrise.minute,
                                                                           hourset = sunset.hour, minset = sunset.minute, 
                                                                           temp = current_temperature, pressure = current_pressure, humidity = current_humidity)
        speak(content)
        time.sleep(20)
    else:
        speak("Không tìm thấy địa chỉ của bạn")

# Chức năng phát nhạc trên Youtube
- Gọi `get_text()` lấy thông tin tên bài hát, lưu vào **mysong**
- Gọi While để tìm kết quả (bỏ qua kết nối yếu, ... )
- Biến **url** lưu đường dẫn kế quả đầu tiên tìm thấy trên Youtube
- Dùng hàm `webbrowser.open(url)` để mở đường dẫn đến video tìm được trên Google Chrome để phát nhạc.

In [35]:
def play_song():
    speak('Xin mời bạn chọn tên bài hát')
    mysong = get_text()
    while True:
        result = YoutubeSearch(mysong, max_results=10).to_dict()
        if result:
            break
    url = 'https://www.youtube.com/watch?v=' + result[0]['id']
    webbrowser.open(url)
    speak("Bài hát bạn yêu cầu đã được mở.")

# 11. Chức năng thay đổi hình nền máy tính
- Dùng **Unsplash** tại trang web `unsplash.com`
- Đăng ký tài khoản để lấy api_key
- Biến **url** lưu đường dẫn đến api của trang web
- Mở url và lấy requests lưu vào biến **json_string**, ảnh sẽ được load, đọc và lưu vào biến **photo**
- **photo** sẽ được lưu lại thành file ảnh trong máy
- Dùng lệnh thay đổi hình nền của máy tính thông qua hàm `ctypes.windll.user32.SystemParametersInfoW()`

In [11]:
def change_wallpaper():
    api_key = 'RF3LyUUIyogjCpQwlf-zjzCf1JdvRwb--SLV6iCzOxw'
    url = 'https://api.unsplash.com/photos/random?client_id=' + \
        api_key  # pic from unspalsh.com
    f = urllib2.urlopen(url)
    json_string = f.read()
    f.close()
    parsed_json = json.loads(json_string)
    photo = parsed_json['urls']['full']
    # Location where we download the image to.
    urllib2.urlretrieve(photo, "C:/Users/Night Fury/Downloads/a.png")
    image=os.path.join("C:/Users/Night Fury/Downloads/a.png")
    ctypes.windll.user32.SystemParametersInfoW(20,0,image,3)
    speak('Hình nền máy tính vừa được thay đổi')

# 12. Chức năng đọc báo ngày hôm nay
- Chọn **Newsapi** tại `newsapi.org`, đăng ký tài khoản lấy apiKey
- Lấy thông tin về chủ đề muốn đọc, dùng requests
- Kết quả thu được kiểu json lưu vào biến **api_response**
- Hiển thị 20 tin tức thu thập và mở đường dẫn đến 3 bài báo đầu tiên.

In [12]:
def read_news():
    speak("Bạn muốn đọc báo về gì?")
    
    queue = get_text()
    params = {
        'apiKey': '30d02d187f7140faacf9ccd27a1441ad',
        "q": queue,
    }
    api_result = requests.get('http://newsapi.org/v2/top-headlines?', params)
    api_response = api_result.json()
    print("Tin tức")

    for number, result in enumerate(api_response['articles'], start=1):
        print(f"""Tin {number}:\nTiêu đề: {result['title']}\nTrích dẫn: {result['description']}\nLink: {result['url']}
    """)
        if number <= 3:
            webbrowser.open(result['url'])

# 13. Chức năng tìm định nghĩa trên từ điển Wikipedia
- Dùng `get_text()` lấy thông tin về thứ muốn định nghĩa, lưu vào **text**
- Gọi hàm `wikipedia.summary(text).split('\n')` để lưu lại thành 1 list các đoạn nội dung mà wikipedia tìm được
- Đọc đoạn định nghĩa đầu tiên
- Máy sẽ hỏi bạn muốn nghe thêm hay không, nếu có máy sẽ đọc tiếp phần tiếp theo, nếu không thì sẽ dừng đọc

In [32]:
def tell_me_about():
    try:
        speak("Bạn muốn nghe về gì?")
        text = get_text()
        contents = wikipedia.summary(text).split('\n')
        speak("Bạn vui lòng chờ một lát")
        speak(contents[0])
        time.sleep(10)
        for content in contents[1:]:
            speak("Bạn muốn nghe thêm không")
            ans = get_text()
            if "có" not in ans:
                break    
            speak(content)
            time.sleep(10)

        speak('Cảm ơn bạn đã lắng nghe!!!')
    except:
        speak("Vietvias không định nghĩa được thuật ngữ của bạn. Xin mời bạn nói lại")

# 14. Chức năng hiển thị các khả năng của trợ lý ảo


In [14]:
def help_me():
    speak("""Vietvias có thể giúp bạn thực hiện các câu lệnh sau đây:
    1. Chào hỏi
    2. Hiển thị giờ
    3. Mở website, application
    4. Tìm kiếm trên Google
    5. Gửi email
    6. Dự báo thời tiết
    7. Mở video nhạc
    8. Thay đổi hình nền máy tính
    9. Đọc báo hôm nay
    10. Kể bạn biết về thế giới """)

# 15. Kết hợp tất cả chức năng Trợ lý ảo Tiếng Việt 
- Dùng các cặp lệnh `if...elif...else` và `while True` để lặp vô hạn cho đến khi bạn muốn dừng
- `get_text()` để lấy thông tin yêu cầu người dùng, lưu vào **text**, nếu **text** nhận 0 tức máy không nghe được, máy sẽ break
- Nếu **text** nhận được giá trị, sẽ thực hiện các câu lệnh, khi nghe thấy "dừng lại" hoặc "tạm biệt" thì cũng sẽ dừng.


In [30]:
def assistant():
    speak("Xin chào, bạn tên là gì?")
    name = get_text()
    if name:
        speak("Chào {}".format(name))
        speak("Vietvias có thể giúp gì?")
        while True:
            text = get_text()
            if not text:
                break
            elif "dừng" in text or "tạm biệt" in text or "chào robot" in text or "ngủ thôi" in text:
                stop()
                break
            elif "chức năng" in text:
                help_me()
            elif "chào hỏi" in text:
                hello(name)
            elif "hiện tại" in text:
                get_time(text)
            elif "mở" in text:
                if 'mở google và tìm kiếm' in text:
                    open_google_and_search(text)
                elif "." in text:
                    open_website(text)
                else:
                    open_application(text)
            elif "email" in text or "mail" in text or "gmail" in text:
                send_email(text)
            elif "thời tiết" in text:
                current_weather()
            elif "chơi nhạc" in text:
                play_song()
            elif "hình nền" in text:
                change_wallpaper()
            elif "đọc báo" in text:
                read_news()
            elif "định nghĩa" in text:
                tell_me_about()
            else:
                speak("Bạn cần Vietvias giúp gì?")

# DEMO 

In [36]:
assistant()

Vietvias: Xin chào, bạn tên là gì?
Tôi: Ronaldo
Vietvias: Chào ronaldo
Vietvias: Vietvias có thể giúp gì?
Tôi: chơi nhạc
Vietvias: Xin mời bạn chọn tên bài hát
Tôi: ...
Vietvias: Vietvias không nghe rõ. Bạn nói lại được không!
Tôi: ok
Vietvias: Bài hát bạn yêu cầu đã được mở.
Tôi: làm đẹp
Vietvias: Bạn cần Vietvias giúp gì?
Tôi: tạm biệt
Vietvias: Tạm biệt
